In [ ]:
from PIL import Image
import imagehash
import os
import matplotlib.pyplot as plt
import numpy as np

data_dir = "data/NBDL"
files = sorted(os.listdir(data_dir), key=lambda x: int(x.split('_')[1].split('.')[0]))

# Tính tất cả Hamming distances giữa các cặp liền kề
hamming_distances = []
pairs = []
for i in range(len(files)-1):
    p1 = os.path.join(data_dir, files[i])
    p2 = os.path.join(data_dir, files[i+1])
    h1 = imagehash.phash(Image.open(p1))
    h2 = imagehash.phash(Image.open(p2))
    d = h1 - h2
    hamming_distances.append(d)
    pairs.append((files[i], files[i+1], d))

hamming_arr = np.array(hamming_distances)

# Vẽ histogram (quan sát phân phối)
plt.figure(figsize=(8,4))
plt.hist(hamming_arr, bins=range(0, max(hamming_arr)+2), edgecolor='k')
plt.xlabel('Hamming distance')
plt.ylabel('Count')
plt.title('Histogram Hamming distances giữa frame liền kề')
plt.show()

# So sánh với vài ngưỡng để biết số cặp "gần giống"
thresholds = [1,2,3,4,5,6,8,10]
for t in thresholds:
    c = np.sum(hamming_arr <= t)
    print(f"<= {t}: {c} cặp ({c/len(hamming_arr):.2%})")

# Loại bỏ ảnh mờ

In [1]:
import cv2
import numpy as np
import os
import fnmatch

In [2]:
PCT = 10  # loại ảnh dưới percentil PCT
PATTERN = "frame_*.jpg"
ROOT_DIR = "data"  # folder gốc chứa nhiều folder con

In [3]:
def var_laplacian_gray(img_path):
    """Trả về variance of Laplacian (float) hoặc None nếu không đọc được."""
    img = cv2.imread(img_path)
    if img is None:
        try:
            img = cv2.imdecode(np.fromfile(img_path, dtype=np.uint8), cv2.IMREAD_COLOR)
        except Exception:
            return None
        if img is None:
            return None
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return float(cv2.Laplacian(gray, cv2.CV_64F).var())

In [4]:
# Lưu kết quả của từng folder
results = {}

In [5]:
for folder in os.listdir(ROOT_DIR):
    folder_path = os.path.join(ROOT_DIR, folder)

    paths = [
        os.path.join(folder_path, f)
        for f in os.listdir(folder_path)
        if fnmatch.fnmatch(f, PATTERN)
    ]
    paths.sort(key=lambda x: int(os.path.basename(x).split("_")[1].split(".")[0]))

    # Tính variance Laplacian
    valid_paths, scores = [], []
    for p in paths:
        v = var_laplacian_gray(p)
        if v is None:
            print(f"[CẢNH BÁO] Không đọc được: {p}")
            continue
        valid_paths.append(p)
        scores.append(v)

    thr = np.percentile(scores, PCT)
    keep_paths = [p for p, s in zip(valid_paths, scores) if s >= thr]

    results[folder] = keep_paths
    print(f"[{folder}] Tổng ảnh hợp lệ: {len(scores)} | Ngưỡng: {thr:.4f} | Giữ lại: {len(keep_paths)} ảnh")


[BMD] Tổng ảnh hợp lệ: 1300 | Ngưỡng: 0.9718 | Giữ lại: 1170 ảnh
[DLDB22] Tổng ảnh hợp lệ: 399 | Ngưỡng: 1.6893 | Giữ lại: 359 ảnh
[DLDB23] Tổng ảnh hợp lệ: 399 | Ngưỡng: 3.9327 | Giữ lại: 359 ảnh
[DLDB24] Tổng ảnh hợp lệ: 399 | Ngưỡng: 2.8725 | Giữ lại: 359 ảnh
[DLDB25] Tổng ảnh hợp lệ: 399 | Ngưỡng: 3.5019 | Giữ lại: 359 ảnh
[DLDB26] Tổng ảnh hợp lệ: 399 | Ngưỡng: 0.9031 | Giữ lại: 359 ảnh
[DuongThu_no_logo] Tổng ảnh hợp lệ: 1000 | Ngưỡng: 0.2472 | Giữ lại: 900 ảnh
[NBDL] Tổng ảnh hợp lệ: 1200 | Ngưỡng: 1.4161 | Giữ lại: 1080 ảnh
[STSH_no_logo] Tổng ảnh hợp lệ: 578 | Ngưỡng: 2.8742 | Giữ lại: 520 ảnh


# Loại bỏ ảnh quá tối/sáng

In [6]:
import shutil

In [7]:
DARK_THR = 25      # median brightness < DARK_THR => quá tối
BRIGHT_THR = 225   # median brightness > BRIGHT_THR => quá sáng
MOVE_REMOVED = True  # True -> di chuyển ảnh bị loại

In [8]:
def _safe_read_bgr(path):
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    if img is None:
        try:
            arr = np.fromfile(path, dtype=np.uint8)
            img = cv2.imdecode(arr, cv2.IMREAD_COLOR)
        except Exception:
            return None
    return img

In [9]:
def brightness_median(path):
    img = _safe_read_bgr(path)
    if img is None:
        return None
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    return float(np.median(hsv[:, :, 2]))


In [10]:
cleaned_results = {}
removed_dark = {} 

In [11]:
for folder, paths in results.items():
    keep, removed = [], []
    for p in paths:
        b = brightness_median(p)
        if b is None:
            print(f"[CẢNH BÁO] Không đọc được ảnh (brightness): {p}")
            continue
        if b < DARK_THR or b > BRIGHT_THR:
            removed.append(p)
        else:
            keep.append(p)
    """
    di chuyển ảnh removed vào 1 folder
    if MOVE_REMOVED and removed:
        for p in removed:
            parent = os.path.dirname(p) or "."
            dest_dir = os.path.join(parent, "removed_dark_bright")
            os.makedirs(dest_dir, exist_ok=True)
            try:
                shutil.move(p, os.path.join(dest_dir, os.path.basename(p)))
            except Exception as e:
                print(f"Không thể di chuyển {p}: {e}")
    Di chuyển ảnh giữ lại vào 1 folder
    if keep:
        for p in keep:
            parent = os.path.dirname(p) or "."
            dest_dir = os.path.join(parent, "keep_bright")
            os.makedirs(dest_dir, exist_ok=True)
            try:
                shutil.move(p, os.path.join(dest_dir, os.path.basename(p)))
            except Exception as e:
                print(f"Không thể di chuyển {p}: {e}")
    """

    cleaned_results[folder] = keep
    print(f"[{folder}] checked={len(paths)} | keep={len(keep)} | removed={len(removed)}")

[BMD] checked=1170 | keep=875 | removed=295
[DLDB22] checked=359 | keep=308 | removed=51
[DLDB23] checked=359 | keep=337 | removed=22
[DLDB24] checked=359 | keep=331 | removed=28
[DLDB25] checked=359 | keep=337 | removed=22
[DLDB26] checked=359 | keep=270 | removed=89
[DuongThu_no_logo] checked=900 | keep=673 | removed=227
[NBDL] checked=1080 | keep=931 | removed=149
[STSH_no_logo] checked=520 | keep=490 | removed=30


# Loại bỏ near-duplicates

In [12]:
from PIL import Image
import imagehash
import os

In [13]:
HAMMING_THR = 10  # ngưỡng Hamming

In [14]:
def phash_imagehash(path, hash_size=8):
    """Trả về imagehash.ImageHash hoặc None nếu không đọc được."""
    try:
        with Image.open(path) as img:
            return imagehash.phash(img, hash_size=hash_size)
    except Exception:
        return None

In [15]:
dedup_results = {}

In [16]:
for folder, paths in cleaned_results.items():
    if not paths:
        dedup_results[folder] = []
        print(f"[{folder}] Không có ảnh để dedup.")
        continue

    kept = []
    last_kept_hash = None
    last_kept_path = None

    for idx, p in enumerate(paths):
        h = phash_imagehash(p)

        if h is None:
            print(f"[CẢNH BÁO] Không đọc được để hash: {p} — giữ ảnh.")
            kept.append(p)
            last_kept_hash = None
            last_kept_path = None
            continue

        if last_kept_hash is None:
            # ảnh đầu tiên hoặc sau ảnh lỗi → giữ
            kept.append(p)
            last_kept_hash = h
            last_kept_path = p
            continue

        dist = last_kept_hash - h
        if dist <= HAMMING_THR:
            print(f"[{folder}] dup: {os.path.basename(last_kept_path)} <-> {os.path.basename(p)} | hamming={dist} -> loại {os.path.basename(p)}")
        else:
            kept.append(p)
            last_kept_hash = h
            last_kept_path = p

    dedup_results[folder] = kept
    print(f"[{folder}] from={len(paths)} kept={len(kept)} removed={len(paths)-len(kept)}")

[BMD] dup: frame_40.jpg <-> frame_41.jpg | hamming=6 -> loại frame_41.jpg
[BMD] dup: frame_40.jpg <-> frame_42.jpg | hamming=10 -> loại frame_42.jpg
[BMD] dup: frame_40.jpg <-> frame_43.jpg | hamming=10 -> loại frame_43.jpg
[BMD] dup: frame_44.jpg <-> frame_45.jpg | hamming=4 -> loại frame_45.jpg
[BMD] dup: frame_44.jpg <-> frame_46.jpg | hamming=8 -> loại frame_46.jpg
[BMD] dup: frame_44.jpg <-> frame_47.jpg | hamming=8 -> loại frame_47.jpg
[BMD] dup: frame_48.jpg <-> frame_49.jpg | hamming=8 -> loại frame_49.jpg
[BMD] dup: frame_50.jpg <-> frame_51.jpg | hamming=10 -> loại frame_51.jpg
[BMD] dup: frame_50.jpg <-> frame_52.jpg | hamming=10 -> loại frame_52.jpg
[BMD] dup: frame_50.jpg <-> frame_53.jpg | hamming=10 -> loại frame_53.jpg
[BMD] dup: frame_54.jpg <-> frame_55.jpg | hamming=6 -> loại frame_55.jpg
[BMD] dup: frame_57.jpg <-> frame_58.jpg | hamming=2 -> loại frame_58.jpg
[BMD] dup: frame_80.jpg <-> frame_81.jpg | hamming=6 -> loại frame_81.jpg
[BMD] dup: frame_83.jpg <-> frame

# Chuẩn bị tập train thành 1 folder chứa ảnh màu và 1 folder chứa ảnh trắng đen

In [17]:
# Thư mục gốc (chứa Color_image và Grayscale_image)
BASE = "cleaned"
COLOR_DIR = os.path.join(BASE, "Color_image")
GRAY_DIR = os.path.join(BASE, "Grayscale_image")

In [18]:
# Tạo các folder đích
os.makedirs(COLOR_DIR, exist_ok=True)
os.makedirs(GRAY_DIR, exist_ok=True)

In [19]:
# dedup_results phải tồn tại: dict {folder_name: [list đường dẫn ảnh, ...], ...}
for folder, paths in dedup_results.items():
    color_subdir = os.path.join(COLOR_DIR, folder)
    gray_subdir = os.path.join(GRAY_DIR, folder)
    os.makedirs(color_subdir, exist_ok=True)
    os.makedirs(gray_subdir, exist_ok=True)

    for img_path in paths:
        if not os.path.isfile(img_path):
            print(f"[CẢNH BÁO] Không tìm thấy file: {img_path}")
            continue

        # copy ảnh màu nguyên bản
        try:
            shutil.copy2(img_path, os.path.join(color_subdir, os.path.basename(img_path)))
        except Exception as e:
            print(f"[LỖI] Không copy {img_path}: {e}")
            continue

        # đọc và chuyển sang grayscale rồi lưu
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        if img is None:
            print(f"[CẢNH BÁO] Không đọc được ảnh để chuyển grayscale: {img_path}")
            continue
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        dest_gray = os.path.join(gray_subdir, os.path.basename(img_path))
        if not cv2.imwrite(dest_gray, gray_img):
            print(f"[LỖI] Không lưu được grayscale: {dest_gray}")

print("✅ Hoàn tất. Color_image và Grayscale_image nằm trong thư mục 'cleaned'.")

✅ Hoàn tất. Color_image và Grayscale_image nằm trong thư mục 'cleaned'.
